# Example Geometry

In [ ]:
import sys, os
sys.path.append(os.path.join(os.path.pardir, 'python'))

In [ ]:
import geometry as geo

In [ ]:
minres = 10.0

In [ ]:
geo.SlabSpline?

In [ ]:
xs = [0.0, 68.2, 154.2, 235, 248, 251, 270, 358, 392]
ys = [-6, -15, -35, -70, -80, -82.5, -100, -200, -240]
res = [1*minres]*(len(xs)-1)+[3*minres]
slab = geo.SlabSpline(xs, ys, res=res, sid=1, name="Slab")
print(len(slab.points))

In [ ]:
geo.SubductionGeometry?

In [ ]:
geom = geo.SubductionGeometry(slab, slab_rid=1, wedge_rid=2, 
                                    slab_side_sid=2, wedge_side_sid=3, slab_base_sid=7, 
                                    wedge_base_sid=8, coast_sid=11, top_sid=12,
                                    wedge_top_res=4*minres, slab_base_res=6*minres)
geom.plot()

In [ ]:
geom.update(coast_distance=260, extra_width=50)
geom.plot()

In [ ]:
geom.addcrustlayer(35,"LowerCrust", sid=9, rid=3, slab_res=1*minres, side_res=4*minres, side_sid=5)
print(len(slab.points))
geom.plot()

In [ ]:
geom.addcrustlayer(15,"UpperCrust", sid=10, rid=4, slab_res= 2*minres, side_res=4*minres, side_sid=6)
print([geom.wedge_side_lines[i].name for i in range(len(geom.wedge_side_lines))])
geom.plot()

In [ ]:
geom.addwedgesidepoint(185, "WedgeSide::InOut", line_name="UpperWedgeSide", res=4*minres, sid=4)
geom.plot()

In [ ]:
mesh, ct, ft = geom.generatemesh()
tdim = mesh.topology.dim
mesh.topology.create_connectivity(tdim, tdim)

In [ ]:
import dolfinx
import numpy as np

V = dolfinx.fem.functionspace(mesh, ("CG", 2))

In [ ]:
all_slab_dofs = dolfinx.fem.locate_dofs_topological(V, 2, ct.find(1))
all_slab_dofs.size

In [ ]:
all_wedge_dofs = dolfinx.fem.locate_dofs_topological(V, 2, ct.find(2))
all_wedge_dofs.size

In [ ]:
slab_surface_dofs = dolfinx.fem.locate_dofs_topological(V, 1, ft.find(1))
slab_surface_dofs.size

In [ ]:
slab_dofs = np.asarray([dof for dof in all_slab_dofs if dof not in slab_surface_dofs])
slab_dofs.size

In [ ]:
wedge_dofs = np.asarray([dof for dof in all_wedge_dofs if dof not in slab_surface_dofs])
wedge_dofs.size